In [ ]:
import gspread
from google.auth import default

# Autenticación con Google (requiere configuración previa de credenciales)
creds, _ = default()
gc = gspread.authorize(creds)

# Reemplaza con la URL o el nombre de tu Google Sheet
sheet_url = 'https://docs.google.com/spreadsheets/d/tu_id_de_hoja_aqui/edit#gid=0'
sh = gc.open_by_url(sheet_url)

# Selecciona la primera hoja
worksheet = sh.get_worksheet(0)

# Lee las primeras 5 filas
data = worksheet.get_all_values()[:5]
for row in data:
    print(row)

: 

In [ ]:
import gspread
from google.auth import default
import pandas as pd

try:
    # Autenticación con Google (requiere configuración previa de credenciales)
    creds, _ = default()
    gc = gspread.authorize(creds)
    
    # URL de tu Google Sheet
    sheet_url = 'https://docs.google.com/spreadsheets/d/1zGyW-M_VV7iyDKVB1TTd0EEP3QBjdoiBmSJN2tK-H7w/edit?resourcekey=&gid=982269766#gid=982269766'
    
    print("Intentando abrir el Google Sheet...")
    sh = gc.open_by_url(sheet_url)
    print(f"✅ Google Sheet abierto exitosamente: {sh.title}")
    
    # Obtener información sobre las hojas disponibles
    worksheets = sh.worksheets()
    print(f"📋 Hojas disponibles: {len(worksheets)}")
    for i, ws in enumerate(worksheets):
        print(f"  {i}: {ws.title} (ID: {ws.id})")
    
    # Seleccionar la hoja específica por su ID (gid=982269766)
    worksheet = None
    for ws in worksheets:
        if ws.id == 982269766:
            worksheet = ws
            break
    
    if worksheet is None:
        print("❌ No se encontró la hoja con ID 982269766, usando la primera hoja disponible")
        worksheet = sh.get_worksheet(0)
    
    print(f"📊 Usando hoja: {worksheet.title}")
    
    # Leer datos de prueba (primeras 10 filas)
    print("\n📖 Leyendo datos...")
    data = worksheet.get_all_values()
    
    if data:
        print(f"✅ Se leyeron {len(data)} filas")
        print("\n🔍 Primeras 5 filas:")
        for i, row in enumerate(data[:5]):
            print(f"Fila {i+1}: {row}")
        
        # Mostrar información adicional
        if len(data) > 1:
            print(f"\n📈 Columnas detectadas: {len(data[0])}")
            print(f"📈 Total de filas con datos: {len(data)}")
    else:
        print("⚠️ La hoja está vacía o no contiene datos")

except gspread.exceptions.SpreadsheetNotFound:
    print("❌ Error: No se pudo encontrar el Google Sheet. Verifica la URL y los permisos.")
except gspread.exceptions.APIError as e:
    print(f"❌ Error de API de Google: {e}")
except Exception as e:
    print(f"❌ Error inesperado: {e}")
    print("💡 Asegúrate de que:")
    print("   - Tienes las credenciales de Google configuradas")
    print("   - El Google Sheet es público o tienes permisos de acceso")
    print("   - Las librerías gspread y google-auth están instaladas")

In [ ]:
import gspread
from google.oauth2.service_account import Credentials
import json
import os

def test_google_sheets_connection():
    """
    Código para probar y diagnosticar la conexión con Google Sheets
    """
    
    print("🔍 DIAGNÓSTICO DE CONEXIÓN GOOGLE SHEETS")
    print("=" * 50)
    
    # 1. Verificar archivo de credenciales - RUTA CORREGIDA
    credentials_file = "../credentials/service_account.json"  # Subir un nivel y buscar credentials
    
    # Si no funciona, intentar rutas alternativas
    alternative_paths = [
        "../credentials/service_account.json",
        "credentials/service_account.json",
        "C:/Users/dell/Desktop/Car/credentials/service_account.json"
    ]
    
    credentials_file = None
    for path in alternative_paths:
        if os.path.exists(path):
            credentials_file = path
            break
    
    print(f"1. Verificando archivo de credenciales...")
    
    if credentials_file is None:
        print("❌ PROBLEMA: Archivo de credenciales no encontrado en ninguna ubicación")
        print("\n📍 Ubicaciones buscadas:")
        for path in alternative_paths:
            abs_path = os.path.abspath(path)
            exists = "✅" if os.path.exists(path) else "❌"
            print(f"   {exists} {abs_path}")
        
        print("\n💡 SOLUCIÓN:")
        print("   Verifica que el archivo esté en: C:/Users/dell/Desktop/Car/credentials/service_account.json")
        return False
    
    print(f"✅ Archivo de credenciales encontrado en: {credentials_file}")
    
    # 2. Verificar formato del archivo de credenciales
    try:
        with open(credentials_file, 'r') as f:
            creds_data = json.load(f)
        
        required_fields = ['type', 'project_id', 'private_key', 'client_email']
        missing_fields = [field for field in required_fields if field not in creds_data]
        
        if missing_fields:
            print(f"❌ PROBLEMA: Campos faltantes en credenciales: {missing_fields}")
            return False
        
        print("✅ Formato de credenciales válido")
        print(f"   Email del servicio: {creds_data['client_email']}")
        print(f"   Proyecto: {creds_data['project_id']}")
        
    except Exception as e:
        print(f"❌ PROBLEMA: Error leyendo credenciales: {e}")
        return False
    
    # 3. Probar autenticación
    try:
        print("\n2. Probando autenticación...")
        
        SCOPES = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive'
        ]
        
        credentials = Credentials.from_service_account_file(
            credentials_file, 
            scopes=SCOPES
        )
        
        gc = gspread.authorize(credentials)
        print("✅ Autenticación exitosa")
        
    except Exception as e:
        print(f"❌ PROBLEMA: Error en autenticación: {e}")
        return False
    
    # 4. Probar acceso al Google Sheet específico
    try:
        print("\n3. Probando acceso al Google Sheet...")
        
        # Tu Google Sheet ID (extraído de la URL)
        sheet_id = '1zGyW-M_VV7iyDKVB1TTd0EEP3QBjdoiBmSJN2tK-H7w'
        
        print(f"   Sheet ID: {sheet_id}")
        
        # Intentar abrir el sheet por ID
        sh = gc.open_by_key(sheet_id)
        print(f"✅ Google Sheet abierto exitosamente: {sh.title}")
        
        # Obtener información de las hojas
        worksheets = sh.worksheets()
        print(f"📋 Hojas disponibles: {len(worksheets)}")
        
        for i, ws in enumerate(worksheets):
            print(f"   {i}: {ws.title} (ID: {ws.id})")
        
        # Buscar la hoja específica (gid=982269766)
        target_worksheet = None
        for ws in worksheets:
            if ws.id == 982269766:
                target_worksheet = ws
                break
        
        if target_worksheet:
            print(f"\n4. Probando lectura de datos de: {target_worksheet.title}")
            
            # Intentar leer algunas celdas
            test_data = target_worksheet.get('A1:C5')
            print(f"✅ Datos leídos exitosamente: {len(test_data)} filas")
            
            if test_data:
                print("📊 Primeras filas:")
                for i, row in enumerate(test_data):
                    print(f"   Fila {i+1}: {row}")
        else:
            print("⚠️ No se encontró la hoja con ID 982269766")
            print("Usando la primera hoja disponible...")
            
            worksheet = sh.get_worksheet(0)
            test_data = worksheet.get('A1:C5')
            print(f"✅ Datos leídos de {worksheet.title}: {len(test_data)} filas")
        
        return True
        
    except gspread.exceptions.SpreadsheetNotFound:
        print("❌ PROBLEMA: Google Sheet no encontrado")
        print("\n💡 POSIBLES CAUSAS:")
        print("   1. La URL del sheet es incorrecta")
        print("   2. El sheet fue eliminado o movido")
        print("   3. No tienes permisos para acceder")
        return False
        
    except gspread.exceptions.APIError as e:
        print(f"❌ PROBLEMA: Error de API de Google: {e}")
        print("\n💡 CAUSAS MÁS COMUNES:")
        print("   1. ❗ NO has compartido el sheet con la cuenta de servicio")
        print("   2. APIs de Google Sheets no están habilitadas")
        print("   3. Límites de cuota excedidos")
        print(f"\n🔑 CUENTA DE SERVICIO: {creds_data['client_email']}")
        print("   ↳ Debes compartir el Google Sheet con este email!")
        return False
        
    except Exception as e:
        print(f"❌ PROBLEMA: Error inesperado: {e}")
        return False

def share_instructions():
    """
    Instrucciones específicas para compartir el Google Sheet
    """
    print("\n🛠️ INSTRUCCIONES PARA COMPARTIR EL GOOGLE SHEET")
    print("=" * 60)
    
    # Encontrar el archivo de credenciales
    alternative_paths = [
        "../credentials/service_account.json",
        "credentials/service_account.json", 
        "C:/Users/dell/Desktop/Car/credentials/service_account.json"
    ]
    
    credentials_file = None
    for path in alternative_paths:
        if os.path.exists(path):
            credentials_file = path
            break
    
    if not credentials_file:
        print("❌ No se pudo encontrar el archivo de credenciales")
        return
    
    # Leer el email de la cuenta de servicio
    try:
        with open(credentials_file, 'r') as f:
            creds_data = json.load(f)
        
        service_email = creds_data['client_email']
        
        print(f"\n📧 EMAIL DE LA CUENTA DE SERVICIO:")
        print(f"   {service_email}")
        
        print(f"\n📋 PASOS PARA COMPARTIR:")
        print(f"   1. Abre este Google Sheet:")
        print(f"      https://docs.google.com/spreadsheets/d/1zGyW-M_VV7iyDKVB1TTd0EEP3QBjdoiBmSJN2tK-H7w/")
        print(f"   ")
        print(f"   2. Haz clic en el botón 'Compartir' (esquina superior derecha)")
        print(f"   ")
        print(f"   3. En 'Agregar personas y grupos', pega este email:")
        print(f"      {service_email}")
        print(f"   ")
        print(f"   4. Selecciona permisos: 'Editor'")
        print(f"   ")
        print(f"   5. Haz clic en 'Enviar'")
        print(f"   ")
        print(f"   6. Vuelve a ejecutar la celda")
        
    except Exception as e:
        print(f"❌ Error leyendo credenciales: {e}")

# Ejecutar la prueba
success = test_google_sheets_connection()

if not success:
    share_instructions()
else:
    print("\n🎉 ¡CONEXIÓN EXITOSA!")
    print("Tu configuración de Google Sheets está funcionando correctamente.")

In [ ]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
import json
import os

def read_google_sheet_with_headers(sheet_id=None, worksheet_name=None, credentials_path=None):
    """
    Lee un Google Sheet usando la primera fila como nombres de columnas
    
    Args:
        sheet_id (str): ID del Google Sheet (opcional, usa el por defecto)
        worksheet_name (str): Nombre de la hoja específica (opcional, usa la primera)
        credentials_path (str): Ruta al archivo de credenciales (opcional)
    
    Returns:
        dict: Diccionario con 'success', 'data', 'columns' y 'message'
    """
    
    # Configuración por defecto
    if sheet_id is None:
        sheet_id = '1zGyW-M_VV7iyDKVB1TTd0EEP3QBjdoiBmSJN2tK-H7w'
    
    # Buscar archivo de credenciales
    if credentials_path is None:
        alternative_paths = [
            "../credentials/service_account.json",
            "credentials/service_account.json",
            "C:/Users/dell/Desktop/Car/credentials/service_account.json"
        ]
        
        for path in alternative_paths:
            if os.path.exists(path):
                credentials_path = path
                break
    
    if not credentials_path:
        return {
            'success': False,
            'data': None,
            'columns': None,
            'message': 'Archivo de credenciales no encontrado'
        }
    
    try:
        # Autenticación
        SCOPES = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive'
        ]
        
        credentials = Credentials.from_service_account_file(
            credentials_path, 
            scopes=SCOPES
        )
        
        gc = gspread.authorize(credentials)
        
        # Abrir el Google Sheet
        sh = gc.open_by_key(sheet_id)
        
        # Seleccionar la hoja de trabajo
        if worksheet_name:
            worksheet = sh.worksheet(worksheet_name)
        else:
            # Buscar por ID específico (gid=982269766) o usar la primera
            worksheets = sh.worksheets()
            worksheet = None
            
            for ws in worksheets:
                if ws.id == 982269766:
                    worksheet = ws
                    break
            
            if worksheet is None:
                worksheet = sh.get_worksheet(0)
        
        # Leer todos los datos
        all_data = worksheet.get_all_values()
        
        if not all_data:
            return {
                'success': False,
                'data': None,
                'columns': None,
                'message': 'La hoja está vacía'
            }
        
        # Primera fila como columnas
        columns = all_data[0]
        data_rows = all_data[1:]  # Resto de filas como datos
        
        # Crear lista de diccionarios (cada fila es un diccionario)
        structured_data = []
        for row in data_rows:
            # Asegurar que la fila tenga la misma longitud que las columnas
            row_data = {}
            for i, column in enumerate(columns):
                value = row[i] if i < len(row) else ''
                row_data[column] = value
            structured_data.append(row_data)
        
        return {
            'success': True,
            'data': structured_data,
            'columns': columns,
            'raw_data': data_rows,
            'total_rows': len(data_rows),
            'sheet_title': sh.title,
            'worksheet_title': worksheet.title,
            'message': f'Datos leídos exitosamente: {len(data_rows)} filas, {len(columns)} columnas'
        }
        
    except gspread.exceptions.SpreadsheetNotFound:
        return {
            'success': False,
            'data': None,
            'columns': None,
            'message': 'Google Sheet no encontrado. Verifica el ID y permisos.'
        }
        
    except gspread.exceptions.APIError as e:
        return {
            'success': False,
            'data': None,
            'columns': None,
            'message': f'Error de API: {e}. Verifica que hayas compartido el sheet con la cuenta de servicio.'
        }
        
    except Exception as e:
        return {
            'success': False,
            'data': None,
            'columns': None,
            'message': f'Error inesperado: {e}'
        }

def create_dataframe_from_sheet(sheet_id=None, worksheet_name=None):
    """
    Crea un DataFrame de pandas desde el Google Sheet
    
    Returns:
        pandas.DataFrame o None si hay error
    """
    result = read_google_sheet_with_headers(sheet_id, worksheet_name)
    
    if result['success']:
        df = pd.DataFrame(result['data'])
        print(f"✅ DataFrame creado: {result['message']}")
        return df
    else:
        print(f"❌ Error: {result['message']}")
        return None

# Función de ejemplo para usar los datos
def example_usage():
    """
    Ejemplo de cómo usar la función
    """
    print("📊 LEYENDO GOOGLE SHEET...")
    print("=" * 50)
    
    # Leer los datos
    result = read_google_sheet_with_headers()
    
    if result['success']:
        print(f"✅ {result['message']}")
        print(f"📋 Hoja: {result['sheet_title']} -> {result['worksheet_title']}")
        print(f"📊 Columnas: {result['columns']}")
        
        # Mostrar primeras 5 filas
        print(f"\n🔍 Primeras 5 filas:")
        for i, row in enumerate(result['data'][:5]):
            print(f"Fila {i+1}: {row}")
        
        # Acceder a datos específicos
        print(f"\n💡 Ejemplos de acceso a datos:")
        if result['data']:
            first_row = result['data'][0]
            print(f"Primera fila completa: {first_row}")
            
            # Acceder por nombre de columna
            for column in result['columns'][:3]:  # Primeras 3 columnas
                if column in first_row:
                    print(f"Columna '{column}': {first_row[column]}")
        
        # Crear DataFrame
        print(f"\n📈 Creando DataFrame...")
        df = create_dataframe_from_sheet()
        if df is not None:
            print(f"DataFrame shape: {df.shape}")
            print(df.head())
        
        return result
    else:
        print(f"❌ {result['message']}")
        return None

# Ejecutar ejemplo
data = example_usage()

In [ ]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
import os

def read_google_sheet_columns():
    """
    Función para leer Google Sheet y mostrar información detallada de las columnas
    """
    
    # Configuración (usa los mismos valores que tu código)
    sheet_id = '1zGyW-M_VV7iyDKVB1TTd0EEP3QBjdoiBmSJN2tK-H7w'
    
    # Buscar credenciales
    credentials_path = None
    alternative_paths = [
        "../credentials/service_account.json",
        "credentials/service_account.json",
        "C:/Users/dell/Desktop/Car/credentials/service_account.json"
    ]
    
    for path in alternative_paths:
        if os.path.exists(path):
            credentials_path = path
            print(f"✅ Credenciales encontradas en: {path}")
            break
    
    if not credentials_path:
        print("❌ Archivo de credenciales no encontrado")
        return None
    
    try:
        # Autenticación
        SCOPES = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive'
        ]
        
        credentials = Credentials.from_service_account_file(
            credentials_path, 
            scopes=SCOPES
        )
        
        gc = gspread.authorize(credentials)
        sh = gc.open_by_key(sheet_id)
        
        # Buscar worksheet específico (gid=982269766) o usar el primero
        worksheets = sh.worksheets()
        worksheet = None
        
        print(f"📊 Google Sheet encontrado: '{sh.title}'")
        print(f"📋 Hojas disponibles:")
        for i, ws in enumerate(worksheets):
            print(f"   {i+1}. '{ws.title}' (ID: {ws.id})")
            if ws.id == 982269766:
                worksheet = ws
                print(f"      ⭐ Esta es la hoja objetivo!")
        
        if worksheet is None:
            worksheet = sh.get_worksheet(0)
            print(f"🔄 Usando la primera hoja: '{worksheet.title}'")
        else:
            print(f"✅ Usando hoja específica: '{worksheet.title}'")
        
        # Leer todos los datos
        all_data = worksheet.get_all_values()
        
        if not all_data:
            print("⚠️ La hoja está vacía")
            return None
        
        # Analizar estructura
        print(f"\n📈 INFORMACIÓN GENERAL:")
        print(f"   • Total de filas (incluyendo header): {len(all_data)}")
        print(f"   • Total de columnas: {len(all_data[0])}")
        print(f"   • Filas de datos (sin header): {len(all_data) - 1}")
        
        # Mostrar columnas (primera fila)
        columns = all_data[0]
        print(f"\n🔍 COLUMNAS ENCONTRADAS ({len(columns)} columnas):")
        for i, col in enumerate(columns, 1):
            print(f"   {i:2d}. '{col}'")
        
        # Crear DataFrame para análisis adicional
        data_rows = all_data[1:]  # Datos sin header
        
        # Crear DataFrame
        structured_data = []
        for row in data_rows:
            row_data = {}
            for i, column in enumerate(columns):
                value = row[i] if i < len(row) else ''
                row_data[column] = value
            structured_data.append(row_data)
        
        df = pd.DataFrame(structured_data)
        
        print(f"\n📊 ANÁLISIS DEL DATAFRAME:")
        print(f"   • Shape: {df.shape}")
        print(f"   • Columnas en DataFrame: {len(df.columns)}")
        print(f"   • Filas con datos: {len(df)}")
        
        # Mostrar información de cada columna
        print(f"\n📋 DETALLE POR COLUMNA:")
        for col in df.columns:
            valores_unicos = df[col].nunique()
            valores_no_vacios = df[col].notna().sum()
            print(f"   📌 '{col}':")
            print(f"      - Valores únicos: {valores_unicos}")
            print(f"      - Valores no vacíos: {valores_no_vacios}")
            print(f"      - Valores vacíos: {len(df) - valores_no_vacios}")
            
            # Mostrar algunos valores de ejemplo
            valores_ejemplo = df[col].dropna().unique()[:3]
            if len(valores_ejemplo) > 0:
                print(f"      - Ejemplos: {list(valores_ejemplo)}")
            print()
        
        return df
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# ==========================================
# EJECUTAR ANÁLISIS
# ==========================================

print("🚀 ANALIZANDO GOOGLE SHEET...")
print("=" * 60)

df = read_google_sheet_columns()

if df is not None:
    print("\n" + "=" * 60)
    print("✅ ANÁLISIS COMPLETADO")
    print("=" * 60)
    
    # Mostrar primeras filas para verificar
    print("\n🔍 PRIMERAS 3 FILAS DE DATOS:")
    display(df.head(3))
    
    # Lista simple de columnas para copiar/pegar
    print(f"\n📝 LISTA DE COLUMNAS (para copiar):")
    print("columnas = [")
    for col in df.columns:
        print(f"    '{col}',")
    print("]")
    
    # Verificar columnas específicas que usa tu código
    print(f"\n🎯 VERIFICACIÓN DE COLUMNAS CLAVE:")
    columnas_esperadas = ['Categoria', 'Severidad de la lesión']
    for col_esperada in columnas_esperadas:
        if col_esperada in df.columns:
            print(f"   ✅ '{col_esperada}' - ENCONTRADA")
        else:
            print(f"   ❌ '{col_esperada}' - NO ENCONTRADA")
            # Buscar columnas similares
            similares = [col for col in df.columns if col_esperada.lower() in col.lower()]
            if similares:
                print(f"      💡 Columnas similares: {similares}")
    
else:
    print("\n❌ No se pudo cargar el DataFrame")

In [ ]:
https://docs.google.com/spreadsheets/d/12SqV7eAYpCwePD-TA1R1XOou-nGO3R6QUSHUnxa8tAI/edit?resourcekey=&gid=382913329#gid=382913329

In [ ]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
import os

def read_new_google_sheet():
    """
    Lee el nuevo Google Sheet con ID: 12SqV7eAYpCwePD-TA1R1XOou-nGO3R6QUSHUnxa8tAI
    y hoja específica gid=382913329
    """
    
    # NUEVO SHEET ID extraído de tu URL
    sheet_id = '12SqV7eAYpCwePD-TA1R1XOou-nGO3R6QUSHUnxa8tAI'
    target_gid = 382913329  # Nuevo gid de la hoja específica
    
    # Buscar credenciales (usando la misma configuración que ya tienes)
    credentials_path = None
    alternative_paths = [
        "../credentials/service_account.json",
        "credentials/service_account.json",
        "C:/Users/dell/Desktop/Car/credentials/service_account.json"
    ]
    
    for path in alternative_paths:
        if os.path.exists(path):
            credentials_path = path
            print(f"✅ Credenciales encontradas en: {path}")
            break
    
    if not credentials_path:
        print("❌ Archivo de credenciales no encontrado")
        return None
    
    try:
        # Autenticación
        SCOPES = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive'
        ]
        
        credentials = Credentials.from_service_account_file(
            credentials_path, 
            scopes=SCOPES
        )
        
        gc = gspread.authorize(credentials)
        
        print(f"🔍 Intentando abrir Google Sheet ID: {sheet_id}")
        sh = gc.open_by_key(sheet_id)
        
        print(f"📊 Google Sheet abierto exitosamente: '{sh.title}'")
        
        # Obtener información sobre las hojas disponibles
        worksheets = sh.worksheets()
        print(f"📋 Hojas disponibles: {len(worksheets)}")
        
        worksheet = None
        for i, ws in enumerate(worksheets):
            print(f"   {i+1}. '{ws.title}' (ID: {ws.id})")
            if ws.id == target_gid:
                worksheet = ws
                print(f"      ⭐ Esta es la hoja objetivo!")
        
        # Si no encontramos la hoja específica, usar la primera
        if worksheet is None:
            worksheet = sh.get_worksheet(0)
            print(f"🔄 No se encontró hoja con gid={target_gid}, usando la primera: '{worksheet.title}'")
        else:
            print(f"✅ Usando hoja específica: '{worksheet.title}'")
        
        # Leer todos los datos
        print(f"\n📖 Leyendo datos de la hoja...")
        all_data = worksheet.get_all_values()
        
        if not all_data:
            print("⚠️ La hoja está vacía")
            return None
        
        # Analizar estructura
        print(f"✅ Datos leídos exitosamente!")
        print(f"   • Total de filas (incluyendo header): {len(all_data)}")
        print(f"   • Total de columnas: {len(all_data[0])}")
        print(f"   • Filas de datos (sin header): {len(all_data) - 1}")
        
        # Primera fila como columnas
        columns = all_data[0]
        data_rows = all_data[1:]  # Resto de filas como datos
        
        print(f"\n🔍 COLUMNAS ENCONTRADAS ({len(columns)} columnas):")
        for i, col in enumerate(columns, 1):
            print(f"   {i:2d}. '{col}'")
        
        # Crear DataFrame
        structured_data = []
        for row in data_rows:
            row_data = {}
            for i, column in enumerate(columns):
                value = row[i] if i < len(row) else ''
                row_data[column] = value
            structured_data.append(row_data)
        
        df = pd.DataFrame(structured_data)
        
        print(f"\n📊 DATAFRAME CREADO:")
        print(f"   • Shape: {df.shape}")
        print(f"   • Columnas: {len(df.columns)}")
        print(f"   • Filas: {len(df)}")
        
        # Mostrar información básica de cada columna
        print(f"\n📋 INFORMACIÓN DE COLUMNAS:")
        for col in df.columns:
            valores_unicos = df[col].nunique()
            valores_no_vacios = df[col].notna().sum()
            print(f"   📌 '{col}': {valores_unicos} únicos, {valores_no_vacios} no vacíos")
        
        # Mostrar primeras filas
        print(f"\n🔍 PRIMERAS 3 FILAS:")
        for i, row in enumerate(df.head(3).to_dict('records')):
            print(f"Fila {i+1}: {row}")
        
        return df
        
    except gspread.exceptions.SpreadsheetNotFound:
        print("❌ ERROR: Google Sheet no encontrado")
        print("💡 POSIBLES CAUSAS:")
        print("   1. El ID del sheet es incorrecto")
        print("   2. El sheet fue eliminado o movido")
        print("   3. No tienes permisos para acceder")
        print("   4. No has compartido el sheet con tu cuenta de servicio")
        
        # Mostrar email de la cuenta de servicio para compartir
        try:
            with open(credentials_path, 'r') as f:
                import json
                creds_data = json.load(f)
                service_email = creds_data['client_email']
                print(f"\n🔑 COMPARTE EL SHEET CON: {service_email}")
        except:
            pass
        
        return None
        
    except gspread.exceptions.APIError as e:
        print(f"❌ ERROR DE API: {e}")
        print("💡 Probablemente necesitas compartir el Google Sheet con tu cuenta de servicio")
        return None
        
    except Exception as e:
        print(f"❌ ERROR INESPERADO: {e}")
        return None

# ==========================================
# EJECUTAR LECTURA DEL NUEVO SHEET
# ==========================================

print("🚀 LEYENDO NUEVO GOOGLE SHEET...")
print("=" * 60)
print("📍 URL: https://docs.google.com/spreadsheets/d/12SqV7eAYpCwePD-TA1R1XOou-nGO3R6QUSHUnxa8tAI/")
print("📍 GID: 382913329")
print("=" * 60)

# Ejecutar la función
df_nuevo = read_new_google_sheet()

if df_nuevo is not None:
    print("\n" + "=" * 60)
    print("✅ ¡ÉXITO! GOOGLE SHEET LEÍDO CORRECTAMENTE")
    print("=" * 60)
    
    # Mostrar el DataFrame completo si es pequeño, o una muestra si es grande
    if len(df_nuevo) <= 10:
        print("\n📊 TODOS LOS DATOS:")
        display(df_nuevo)
    else:
        print(f"\n📊 MUESTRA DE DATOS (primeras 5 filas de {len(df_nuevo)} totales):")
        display(df_nuevo.head())
    
    # Guardar en variable global para usar después
    print(f"\n💾 DataFrame guardado en variable 'df_nuevo'")
    print(f"📈 Puedes usarlo con: df_nuevo.head(), df_nuevo.info(), etc.")
    
else:
    print("\n❌ NO SE PUDO LEER EL GOOGLE SHEET")
    print("🔧 Verifica que hayas compartido el sheet con tu cuenta de servicio")

In [ ]:
# Ya tienes la función read_new_google_sheet() definida arriba y df_nuevo como variable.
# Si quieres asegurarte de tener el DataFrame actualizado con los datos del Google Sheet, simplemente vuelve a ejecutar:

df_nuevo = read_new_google_sheet()

# Ahora df_nuevo contendrá los datos del Google Sheet como un DataFrame de pandas listo para trabajar.
# Ejemplo de uso:
print(df_nuevo.head())

In [ ]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
import os

def read_new_google_sheet_to_df(
    sheet_id='12SqV7eAYpCwePD-TA1R1XOou-nGO3R6QUSHUnxa8tAI',
    target_gid=382913329,
    credentials_paths=None
):
    """
    Lee un Google Sheet específico y devuelve un DataFrame de pandas.
    Args:
        sheet_id (str): ID del Google Sheet.
        target_gid (int): GID de la hoja específica.
        credentials_paths (list): Lista de rutas posibles para el archivo de credenciales.
    Returns:
        pd.DataFrame o None si hay error.
    """
    if credentials_paths is None:
        credentials_paths = [
            "../credentials/service_account.json",
            "credentials/service_account.json",
            "C:/Users/dell/Desktop/Car/credentials/service_account.json"
        ]
    credentials_path = None
    for path in credentials_paths:
        if os.path.exists(path):
            credentials_path = path
            break
    if not credentials_path:
        print("❌ Archivo de credenciales no encontrado")
        return None

    try:
        SCOPES = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive'
        ]
        credentials = Credentials.from_service_account_file(
            credentials_path, 
            scopes=SCOPES
        )
        gc = gspread.authorize(credentials)
        sh = gc.open_by_key(sheet_id)
        worksheets = sh.worksheets()
        worksheet = None
        for ws in worksheets:
            if ws.id == target_gid:
                worksheet = ws
                break
        if worksheet is None:
            worksheet = sh.get_worksheet(0)
        all_data = worksheet.get_all_values()
        if not all_data:
            print("⚠️ La hoja está vacía")
            return None
        columns = all_data[0]
        data_rows = all_data[1:]
        structured_data = []
        for row in data_rows:
            row_data = {}
            for i, column in enumerate(columns):
                value = row[i] if i < len(row) else ''
                row_data[column] = value
            structured_data.append(row_data)
        df = pd.DataFrame(structured_data)
        return df
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Ejemplo de uso:
if __name__ == "__main__":
    df_nuevo = read_new_google_sheet_to_df()
    if df_nuevo is not None:
        print(df_nuevo.head())
    else:
        print("No se pudo leer el Google Sheet.")

In [ ]:
print(df_nuevo.columns.tolist())

In [ ]:
def contar_jugadores_por_categoria(df=None):
    """
    Cuenta cuántos jugadores hay medidos por cada categoría.
    
    Args:
        df (pd.DataFrame): DataFrame con los datos. Si es None, usa df_nuevo.
    
    Returns:
        pd.Series: Serie con el conteo por categoría
    """
    # Si no se pasa DataFrame, usar df_nuevo
    if df is None:
        df = df_nuevo
    
    # Verificar que el DataFrame no esté vacío
    if df is None or df.empty:
        print("❌ Error: DataFrame vacío o no existe")
        return None
    
    # Verificar que existe la columna 'Categoría'
    if 'Categoría' not in df.columns:
        print("❌ Error: No se encontró la columna 'Categoría'")
        print(f"📋 Columnas disponibles: {list(df.columns)}")
        return None
    
    # Contar jugadores por categoría
    conteo = df['Categoría'].value_counts()
    
    # Mostrar resultados
    print("📊 JUGADORES MEDIDOS POR CATEGORÍA")
    print("=" * 40)
    
    total_jugadores = conteo.sum()
    print(f"🏃 Total de jugadores medidos: {total_jugadores}")
    print()
    
    for categoria, cantidad in conteo.items():
        porcentaje = (cantidad / total_jugadores) * 100
        print(f"🏆 {categoria}: {cantidad} jugadores ({porcentaje:.1f}%)")
    
    return conteo

# Función adicional para mostrar más detalles
def analisis_completo_categorias(df=None):
    """
    Análisis más detallado de las categorías
    """
    if df is None:
        df = df_nuevo
    
    if df is None or df.empty:
        print("❌ Error: DataFrame vacío o no existe")
        return None
    
    if 'Categoría' not in df.columns:
        print("❌ Error: No se encontró la columna 'Categoría'")
        return None
    
    print("📈 ANÁLISIS DETALLADO POR CATEGORÍAS")
    print("=" * 50)
    
    # Conteo básico
    conteo = df['Categoría'].value_counts()
    
    # Información general
    print(f"📊 Total de categorías diferentes: {len(conteo)}")
    print(f"🏃 Total de jugadores medidos: {conteo.sum()}")
    print(f"📋 Categorías: {list(conteo.index)}")
    print()
    
    # Detalles por categoría
    print("🔍 DETALLE POR CATEGORÍA:")
    for i, (categoria, cantidad) in enumerate(conteo.items(), 1):
        porcentaje = (cantidad / conteo.sum()) * 100
        print(f"   {i}. {categoria}: {cantidad} jugadores ({porcentaje:.1f}%)")
    
    # Categoría con más y menos jugadores
    print(f"\n🥇 Categoría con más jugadores: {conteo.index[0]} ({conteo.iloc[0]} jugadores)")
    print(f"🥉 Categoría con menos jugadores: {conteo.index[-1]} ({conteo.iloc[-1]} jugadores)")
    
    return conteo

# EJECUTAR LAS FUNCIONES
print("🚀 ANALIZANDO JUGADORES POR CATEGORÍA...")
print()

# Función simple
resultado = contar_jugadores_por_categoria()

print("\n" + "=" * 50)

# Análisis completo
analisis_completo_categorias()

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
from datetime import datetime
from google.oauth2.service_account import Credentials
from plotly.subplots import make_subplots
import gspread

def read_new_google_sheet_to_df(
    sheet_id='12SqV7eAYpCwePD-TA1R1XOou-nGO3R6QUSHUnxa8tAI',
    target_gid=382913329,
    credentials_paths=None
):
    """
    Lee un Google Sheet específico y devuelve un DataFrame de pandas.
    """
    if credentials_paths is None:
        credentials_paths = [
            "../credentials/service_account.json",
            "credentials/service_account.json",
            "C:/Users/dell/Desktop/Car/credentials/service_account.json",
            "service_account.json"
        ]
    
    credentials_path = None
    for path in credentials_paths:
        if os.path.exists(path):
            credentials_path = path
            break
    
    if not credentials_path:
        st.error(f"❌ Archivo de credenciales no encontrado en las rutas: {credentials_paths}")
        st.info("🔧 Verifica que el archivo service_account.json esté en una de estas ubicaciones")
        return None

    try:
        SCOPES = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive'
        ]
        credentials = Credentials.from_service_account_file(
            credentials_path, 
            scopes=SCOPES
        )
        gc = gspread.authorize(credentials)
        sh = gc.open_by_key(sheet_id)
        
        # Obtener la hoja de trabajo específica por GID
        worksheet = None
        for sheet in sh.worksheets():
            if str(sheet.id) == str(target_gid):
                worksheet = sheet
                break
        
        if worksheet is None:
            st.error(f"❌ No se encontró la hoja con GID: {target_gid}")
            return None
        
        # Leer todos los datos
        data = worksheet.get_all_records()
        
        if not data:
            st.warning("⚠️ La hoja está vacía")
            return pd.DataFrame()
        
        # Convertir a DataFrame
        df = pd.DataFrame(data)
        
        # Limpiar datos vacíos
        df = df.dropna(how='all')
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
        
        return df
        
    except FileNotFoundError:
        st.error("❌ Archivo de credenciales no encontrado")
        return None
    except Exception as e:
        st.error(f"❌ Error al conectar con Google Sheets: {str(e)}")
        st.info("🔧 Verifica que:")
        st.info("• El archivo de credenciales sea válido")
        st.info("• El Sheet ID sea correcto")
        st.info("• La cuenta de servicio tenga permisos")
        return None

def crear_header_profesional():
    """Crea un header profesional similar al área médica"""
    # CSS personalizado para el header
    st.markdown("""
    <style>
    .nutrition-header {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 2rem;
        border-radius: 15px;
        margin-bottom: 2rem;
        box-shadow: 0 10px 30px rgba(0,0,0,0.1);
        text-align: center;
    }
    .nutrition-title {
        color: white;
        font-size: 2.5rem;
        font-weight: 700;
        margin: 0;
        text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
    }
    .nutrition-subtitle {
        color: rgba(255,255,255,0.9);
        font-size: 1.2rem;
        margin: 0.5rem 0 0 0;
        font-weight: 300;
    }
    .professional-metric {
        background: white;
        padding: 1.5rem;
        border-radius: 10px;
        box-shadow: 0 4px 15px rgba(0,0,0,0.1);
        border-left: 4px solid #667eea;
        margin: 0.5rem 0;
    }
    .metric-title {
        color: #667eea;
        font-size: 0.9rem;
        font-weight: 600;
        margin-bottom: 0.5rem;
        text-transform: uppercase;
        letter-spacing: 0.5px;
    }
    .metric-value {
        color: #2c3e50;
        font-size: 2rem;
        font-weight: 700;
        margin: 0;
    }
    .metric-delta {
        color: #7f8c8d;
        font-size: 0.9rem;
        margin-top: 0.3rem;
    }
    .section-card {
        background: white;
        padding: 1.5rem;
        border-radius: 12px;
        box-shadow: 0 2px 10px rgba(0,0,0,0.05);
        border: 1px solid #e9ecef;
        margin: 1rem 0;
    }
    .section-title {
        color: #667eea;
        font-size: 1.3rem;
        font-weight: 600;
        margin-bottom: 1rem;
        display: flex;
        align-items: center;
        gap: 0.5rem;
    }
    </style>
    """, unsafe_allow_html=True)
    
    # Header principal
    st.markdown("""
    <div class="nutrition-header">
        <h1 class="nutrition-title">🥗 ÁREA DE NUTRICIÓN</h1>
        <p class="nutrition-subtitle">Sistema Profesional de Análisis Nutricional</p>
        <p class="nutrition-subtitle">Club Argentino de Rugby</p>
    </div>
    """, unsafe_allow_html=True)

def crear_selector_categorias(df):
    """Crea un selector múltiple para elegir categorías específicas."""
    if df is None or df.empty:
        st.error("❌ Error: DataFrame vacío o no existe")
        return []
    
    if 'Categoría' not in df.columns:
        st.error("❌ Error: No se encontró la columna 'Categoría'")
        return []
    
    # Obtener categorías únicas disponibles
    categorias_disponibles = sorted(df['Categoría'].unique())
    categorias_disponibles = [cat for cat in categorias_disponibles if cat.strip()]
    
    st.markdown('<div class="section-card">', unsafe_allow_html=True)
    st.markdown('<h3 class="section-title">🎯 Seleccionar Categorías</h3>', unsafe_allow_html=True)
    
    # Opciones para seleccionar todas o ninguna
    col1, col2 = st.columns(2)
    with col1:
        seleccionar_todas = st.button("✅ Seleccionar Todas", key="sel_todas_cat")
    with col2:
        limpiar_seleccion = st.button("❌ Limpiar Selección", key="limpiar_cat")
    
    # Selector múltiple principal
    if seleccionar_todas:
        categorias_seleccionadas = st.multiselect(
            "Elige las categorías a analizar:",
            options=categorias_disponibles,
            default=categorias_disponibles,
            help="Puedes seleccionar múltiples categorías para comparar"
        )
    elif limpiar_seleccion:
        categorias_seleccionadas = st.multiselect(
            "Elige las categorías a analizar:",
            options=categorias_disponibles,
            default=[],
            help="Puedes seleccionar múltiples categorías para comparar"
        )
    else:
        categorias_seleccionadas = st.multiselect(
            "Elige las categorías a analizar:",
            options=categorias_disponibles,
            default=categorias_disponibles[:3] if len(categorias_disponibles) >= 3 else categorias_disponibles,
            help="Puedes seleccionar múltiples categorías para comparar"
        )
    
    st.markdown('</div>', unsafe_allow_html=True)
    return categorias_seleccionadas

def crear_selector_jugadores(df, categorias_seleccionadas):
    """Crea un selector múltiple para elegir jugadores específicos."""
    if df is None or df.empty:
        st.error("❌ Error: DataFrame vacío o no existe")
        return []
    
    if not categorias_seleccionadas:
        st.info("👈 Primero selecciona una categoría")
        return []
    
    columna_jugador = 'Nombre y Apellido'
    
    if columna_jugador not in df.columns:
        st.error(f"❌ Error: No se encontró la columna '{columna_jugador}'")
        return []
    
    df_filtrado = df[df['Categoría'].isin(categorias_seleccionadas)]
    
    if df_filtrado.empty:
        st.warning("⚠️ No hay jugadores en las categorías seleccionadas")
        return []
    
    jugadores_disponibles = df_filtrado[columna_jugador].dropna().unique()
    jugadores_disponibles = [str(jug) for jug in jugadores_disponibles if str(jug).strip() and str(jug) != 'nan']
    jugadores_disponibles = sorted(jugadores_disponibles)
    
    if not jugadores_disponibles:
        st.warning("⚠️ No hay jugadores disponibles en las categorías seleccionadas")
        return []
    
    st.markdown('<div class="section-card">', unsafe_allow_html=True)
    st.markdown('<h3 class="section-title">👤 Seleccionar Jugadores</h3>', unsafe_allow_html=True)
    
    # Opciones para seleccionar todos o ninguno
    col1, col2 = st.columns(2)
    with col1:
        seleccionar_todos = st.button("✅ Seleccionar Todos", key="sel_todos_jug")
    with col2:
        limpiar_jugadores = st.button("❌ Limpiar Selección", key="limpiar_jug")
    
    # Selector múltiple principal
    if seleccionar_todos:
        jugadores_seleccionados = st.multiselect(
            f"Jugadores disponibles ({len(jugadores_disponibles)}):",
            options=jugadores_disponibles,
            default=jugadores_disponibles,
            help="Selecciona jugadores para análisis detallado"
        )
    elif limpiar_jugadores:
        jugadores_seleccionados = st.multiselect(
            f"Jugadores disponibles ({len(jugadores_disponibles)}):",
            options=jugadores_disponibles,
            default=[],
            help="Selecciona jugadores para análisis detallado"
        )
    else:
        jugadores_seleccionados = st.multiselect(
            f"Jugadores disponibles ({len(jugadores_disponibles)}):",
            options=jugadores_disponibles,
            default=jugadores_disponibles[:5] if len(jugadores_disponibles) >= 5 else jugadores_disponibles,
            help="Selecciona jugadores para análisis detallado"
        )
    
    st.markdown('</div>', unsafe_allow_html=True)
    return jugadores_seleccionados

def analizar_objetivos_por_categoria(df, categorias_seleccionadas):
    """Analiza los objetivos nutricionales por categoría"""
    if df is None or df.empty or not categorias_seleccionadas:
        return None
    
    if 'Objetivo' not in df.columns:
        st.warning("⚠️ No se encontró la columna 'Objetivo'")
        return None
    
    # Filtrar por categorías seleccionadas
    df_filtrado = df[df['Categoría'].isin(categorias_seleccionadas)]
    
    if df_filtrado.empty:
        return None
    
    # Análisis por categoría
    resultados = {}
    for categoria in categorias_seleccionadas:
        df_cat = df_filtrado[df_filtrado['Categoría'] == categoria]
        if not df_cat.empty:
            objetivos_count = df_cat['Objetivo'].value_counts()
            resultados[categoria] = objetivos_count
    
    return resultados

def crear_metricas_profesionales(categorias_seleccionadas, conteo_categorias, jugadores_seleccionados):
    """Crea métricas con estilo profesional"""
    st.markdown('<div class="section-card">', unsafe_allow_html=True)
    st.markdown('<h3 class="section-title">📊 Panel de Control Nutricional</h3>', unsafe_allow_html=True)
    
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        st.markdown(f"""
        <div class="professional-metric">
            <div class="metric-title">Categorías Activas</div>
            <div class="metric-value">{len(categorias_seleccionadas)}</div>
            <div class="metric-delta">En análisis</div>
        </div>
        """, unsafe_allow_html=True)
    
    with col2:
        total_jugadores = conteo_categorias.sum() if conteo_categorias is not None else 0
        st.markdown(f"""
        <div class="professional-metric">
            <div class="metric-title">Total Atletas</div>
            <div class="metric-value">{total_jugadores}</div>
            <div class="metric-delta">Con evaluación nutricional</div>
        </div>
        """, unsafe_allow_html=True)
    
    with col3:
        jugadores_sel = len(jugadores_seleccionados) if jugadores_seleccionados else 0
        st.markdown(f"""
        <div class="professional-metric">
            <div class="metric-title">Atletas Seleccionados</div>
            <div class="metric-value">{jugadores_sel}</div>
            <div class="metric-delta">Para análisis detallado</div>
        </div>
        """, unsafe_allow_html=True)
    
    with col4:
        porcentaje = (jugadores_sel / total_jugadores * 100) if total_jugadores > 0 else 0
        st.markdown(f"""
        <div class="professional-metric">
            <div class="metric-title">Cobertura Análisis</div>
            <div class="metric-value">{porcentaje:.1f}%</div>
            <div class="metric-delta">Del total seleccionado</div>
        </div>
        """, unsafe_allow_html=True)
    
    st.markdown('</div>', unsafe_allow_html=True)

def crear_grafico_objetivos(resultados_objetivos):
    """Crea gráficos para mostrar la distribución de objetivos"""
    if not resultados_objetivos:
        return None, None
    
    # Preparar datos para el gráfico
    categorias = []
    objetivos_data = {}
    
    for categoria, objetivos_count in resultados_objetivos.items():
        categorias.append(categoria)
        for objetivo, count in objetivos_count.items():
            if objetivo not in objetivos_data:
                objetivos_data[objetivo] = []
            objetivos_data[objetivo].append(count)
        
        # Asegurar que todos los objetivos tengan valores para todas las categorías
        for objetivo in objetivos_data:
            if len(objetivos_data[objetivo]) < len(categorias):
                objetivos_data[objetivo].append(0)
    
    # Gráfico de barras agrupadas
    fig_barras = go.Figure()
    
    colores = {
        'Mantenimiento de peso corporal': '#3498db',
        'Aumento de Masa Muscular': '#e74c3c', 
        'Disminución de Masa Adiposa': '#f39c12'
    }
    
    for objetivo, valores in objetivos_data.items():
        color = colores.get(objetivo, '#95a5a6')
        fig_barras.add_trace(go.Bar(
            name=objetivo,
            x=categorias,
            y=valores,
            marker_color=color,
            text=valores,
            textposition='auto'
        ))
    
    fig_barras.update_layout(
        title="📈 Distribución de Objetivos Nutricionales por Categoría",
        xaxis_title="Categoría",
        yaxis_title="Número de Atletas",
        barmode='group',
        height=400,
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    # Gráfico de dona para objetivos generales
    todos_objetivos = {}
    for objetivos_count in resultados_objetivos.values():
        for objetivo, count in objetivos_count.items():
            todos_objetivos[objetivo] = todos_objetivos.get(objetivo, 0) + count
    
    fig_dona = go.Figure(data=[go.Pie(
        labels=list(todos_objetivos.keys()),
        values=list(todos_objetivos.values()),
        hole=0.4,
        marker=dict(colors=[colores.get(obj, '#95a5a6') for obj in todos_objetivos.keys()])
    )])
    
    fig_dona.update_layout(
        title="🎯 Distribución General de Objetivos",
        height=400,
        showlegend=True,
        legend=dict(orientation="v", yanchor="middle", y=0.5, xanchor="left", x=1.05)
    )
    
    return fig_barras, fig_dona

def mostrar_detalle_por_categoria(df, categorias_seleccionadas, resultados_objetivos):
    """Muestra el detalle profesional por cada categoría"""
    if not categorias_seleccionadas or not resultados_objetivos:
        return
    
    st.markdown('<div class="section-card">', unsafe_allow_html=True)
    st.markdown('<h3 class="section-title">📋 Análisis Detallado por Categoría</h3>', unsafe_allow_html=True)
    
    for categoria in categorias_seleccionadas:
        if categoria in resultados_objetivos:
            objetivos_count = resultados_objetivos[categoria]
            total_categoria = objetivos_count.sum()
            
            st.markdown(f"### 🏆 {categoria}")
            st.markdown(f"**Total de atletas:** {total_categoria}")
            
            # Mostrar objetivos en columnas
            col1, col2, col3 = st.columns(3)
            
            objetivos = ['Mantenimiento de peso corporal', 'Aumento de Masa Muscular', 'Disminución de Masa Adiposa']
            colores = ['🔵', '🔴', '🟡']
            columnas = [col1, col2, col3]
            
            for i, objetivo in enumerate(objetivos):
                with columnas[i]:
                    cantidad = objetivos_count.get(objetivo, 0)
                    porcentaje = (cantidad / total_categoria * 100) if total_categoria > 0 else 0
                    
                    st.markdown(f"""
                    <div style="
                        background: white;
                        padding: 1rem;
                        border-radius: 8px;
                        border-left: 4px solid #667eea;
                        margin: 0.5rem 0;
                        box-shadow: 0 2px 8px rgba(0,0,0,0.05);
                    ">
                        <div style="color: #667eea; font-size: 0.8rem; font-weight: 600; margin-bottom: 0.3rem;">
                            {colores[i]} {objetivo.upper()}
                        </div>
                        <div style="font-size: 1.5rem; font-weight: 700; color: #2c3e50;">
                            {cantidad}
                        </div>
                        <div style="color: #7f8c8d; font-size: 0.9rem;">
                            {porcentaje:.1f}% del total
                        </div>
                    </div>
                    """, unsafe_allow_html=True)
            
            st.markdown("---")
    
    st.markdown('</div>', unsafe_allow_html=True)

def mostrar_analisis_nutricion():
    """Función principal para mostrar el análisis de nutrición"""
    # Header profesional
    crear_header_profesional()
    
    # Cargar datos
    with st.spinner("📊 Cargando datos nutricionales desde Google Sheets..."):
        df = read_new_google_sheet_to_df()
    
    if df is None:
        st.error("❌ No se pudieron cargar los datos nutricionales")
        st.info("🔧 Verifica que las credenciales estén configuradas correctamente")
        st.stop()
    
    if df.empty:
        st.warning("⚠️ No hay datos nutricionales disponibles")
        st.stop()
    
    # Estado de carga exitosa
    st.success(f"✅ **{len(df)} atletas** con evaluaciones nutricionales cargados exitosamente")
    
    # Selectores en dos columnas
    col_izq, col_der = st.columns(2)
    
    with col_izq:
        categorias_seleccionadas = crear_selector_categorias(df)
    
    with col_der:
        jugadores_seleccionados = crear_selector_jugadores(df, categorias_seleccionadas)
    
    # Análisis principal
    if categorias_seleccionadas:
        st.markdown("---")
        
        # Contar jugadores por categoría
        df_filtrado = df[df['Categoría'].isin(categorias_seleccionadas)]
        conteo_categorias = df_filtrado['Categoría'].value_counts()
        
        # Métricas profesionales
        crear_metricas_profesionales(categorias_seleccionadas, conteo_categorias, jugadores_seleccionados)
        
        # Análisis de objetivos
        resultados_objetivos = analizar_objetivos_por_categoria(df, categorias_seleccionadas)
        
        if resultados_objetivos:
            # Detalle por categoría
            mostrar_detalle_por_categoria(df, categorias_seleccionadas, resultados_objetivos)
            
            # Gráficos de objetivos
            fig_barras, fig_dona = crear_grafico_objetivos(resultados_objetivos)
            
            if fig_barras and fig_dona:
                st.markdown('<div class="section-card">', unsafe_allow_html=True)
                st.markdown('<h3 class="section-title">📈 Visualización de Objetivos</h3>', unsafe_allow_html=True)
                
                col1, col2 = st.columns(2)
                with col1:
                    st.plotly_chart(fig_barras, use_container_width=True)
                with col2:
                    st.plotly_chart(fig_dona, use_container_width=True)
                
                st.markdown('</div>', unsafe_allow_html=True)
        
        # Tabla de jugadores seleccionados
        if jugadores_seleccionados:
            st.markdown('<div class="section-card">', unsafe_allow_html=True)
            st.markdown('<h3 class="section-title">👤 Atletas Seleccionados - Análisis Detallado</h3>', unsafe_allow_html=True)
            
            columna_jugador = 'Nombre y Apellido'
            if columna_jugador in df.columns:
                df_jugadores = df[df[columna_jugador].isin(jugadores_seleccionados)]
                
                cols_mostrar = [columna_jugador, 'Categoría', 'Objetivo']
                cols_adicionales = [
                    'Posición del jugador', 
                    'Peso (kg): [Número con decimales 88,5]', 
                    'Talla (cm): [Número]', 
                    'IMC',
                    'Cuantos kilos de  Masa Muscular'
                ]
                
                for col in cols_adicionales:
                    if col in df.columns:
                        cols_mostrar.append(col)
                
                st.dataframe(
                    df_jugadores[cols_mostrar].reset_index(drop=True),
                    use_container_width=True
                )
            
            st.markdown('</div>', unsafe_allow_html=True)
    else:
        st.info("👆 Selecciona al menos una categoría para comenzar el análisis nutricional")

In [ ]:
import streamlit as st
import pandas as pd

st.set_page_config(page_title="Análisis Nutricional", layout="wide")

st.title("📊 Análisis Nutricional - Club Argentino de Rugby")

st.markdown("### Datos cargados desde Google Sheets")

# Mostrar el DataFrame
st.dataframe(df_nuevo, use_container_width=True)

# Mostrar conteo por categoría si existe la variable resultado
if 'resultado' in locals():
    st.markdown("### Conteo de jugadores por Categoría")
    st.bar_chart(resultado)
else:
    st.info("No se encontró la variable 'resultado'. Ejecuta el análisis primero.")

# Puedes agregar más visualizaciones aquí según lo necesites

In [ ]:
def grafico_evolucion_peso(df_hist):
    """
    Devuelve un gráfico de línea suavizada (spline) con la evolución del peso del jugador,
    estilo Power BI oscuro y minimalista.
    """
    columna_fecha = obtener_columna_fecha(df_hist)
    columna_peso = 'Peso (kg): [Número con decimales 88,5]'
    if columna_fecha and columna_peso in df_hist.columns:
        df_hist_ordenado = df_hist.sort_values(columna_fecha)
        # Convertir columna de fecha a datetime si no lo está
        if not pd.api.types.is_datetime64_any_dtype(df_hist_ordenado[columna_fecha]):
            df_hist_ordenado[columna_fecha] = pd.to_datetime(df_hist_ordenado[columna_fecha], errors='coerce')
        # Formatear fechas para el hover (mes abreviado + año)
        fechas_formateadas = df_hist_ordenado[columna_fecha].dt.strftime('%b %Y')
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=df_hist_ordenado[columna_fecha],
            y=df_hist_ordenado[columna_peso],
            mode='lines+markers',
            line=dict(
                shape='spline',
                color='#f2c94c',
                width=3
            ),
            marker=dict(
                color='#f2c94c',
                size=10,
                line=dict(color='white', width=2)
            ),
            hovertemplate=(
                "<b>Peso:</b> %{y:.1f} kg<br>" +
                "<b>Fecha:</b> %{customdata}<extra></extra>"
            ),
            customdata=fechas_formateadas
        ))
        fig.update_layout(
            title=dict(
                text="Evolución del Peso (kg)",
                x=0.5,
                font=dict(color='white', size=22, family='Arial')
            ),
            plot_bgcolor='#1a1a1a',
            paper_bgcolor='#1a1a1a',
            xaxis=dict(
                title=dict(
                    text="Fecha",
                    font=dict(color='white', size=16)
                ),
                tickfont=dict(color='white', size=14),
                gridcolor='rgba(255,255,255,0.1)',
                showline=False,
                zeroline=False
            ),
            yaxis=dict(
                title=dict(
                    text="Peso (kg)",
                    font=dict(color='white', size=16)
                ),
                tickfont=dict(color='white', size=14),
                gridcolor='rgba(255,255,255,0.1)',
                showline=False,
                zeroline=False
            ),
            font=dict(color='white'),
            margin=dict(l=40, r=40, t=60, b=40),
            height=350,
            showlegend=False
        )
        return fig
    else:
        return None
   

In [ ]:
import pandas as pd

# Supongamos que tienes un DataFrame de ejemplo con fechas y pesos para probar la función

# Crear un DataFrame de ejemplo
df_hist = pd.DataFrame({
    'Fecha de medición': pd.date_range(start='2023-01-01', periods=6, freq='M'),
    'Peso (kg): [Número con decimales 88,5]': [88.5, 89.2, 87.8, 88.0, 89.0, 88.7]
})

# Función auxiliar para detectar la columna de fecha (puedes adaptarla si tienes otra lógica)
def obtener_columna_fecha(df):
    for col in df.columns:
        if 'fecha' in col.lower():
            return col
    return None

# Importar plotly para mostrar el gráfico
import plotly.graph_objects as go

# Llamar a la función y mostrar el gráfico
fig = grafico_evolucion_peso(df_hist)
if fig:
    fig.show()
else:
# Importar librerías necesarias
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
import os
import json
from IPython.display import display
import plotly.graph_objects as go    print("No se pudo generar el gráfico. Verifica los nombres de las columnas.")

In [ ]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
import os
import json
from IPython.display import display
import plotly.graph_objects as go


# Conectar el Google Sheet indicado y obtener un DataFrame
sheet_id = '180ikmYPmc1nxw5UZYFq9lDa0lGfLn_L-7Yb8CmwJAPM'
target_gid = 668185687

print("🔍 Intentando leer Google Sheet:", sheet_id, "gid:", target_gid)

# Reusar la función existente read_new_google_sheet_to_df definida en celdas anteriores
df_sheet_180 = read_new_google_sheet_to_df(sheet_id=sheet_id, target_gid=target_gid)

if df_sheet_180 is None:
    print("❌ No se pudo leer el Google Sheet. Revisa credenciales/permisos.")
else:
    print(f"✅ DataFrame cargado: {df_sheet_180.shape[0]} filas x {df_sheet_180.shape[1]} columnas")
    display(df_sheet_180.head())
    print("\nColumnas:")
    print(df_sheet_180.columns.tolist())

: 